In [4]:
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, HernquistPotential
from galpy.actionAngle import estimateDeltaStaeckel, actionAngleStaeckel
from galpy.actionAngle import UnboundError

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

import numpy as np
from scipy import stats
from IPython.display import Image, display

import corner

import datetime
import random

import copy

import struct

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Circle
from matplotlib import animation
from matplotlib.colors import LogNorm


import datetime

cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))

%matplotlib inline

In [2]:
# Simulation relevant stuff
machine = 'mac'
machine = 'magny'

if machine == 'magny':
    filedir = "/home/extmilan/masterthesis/files/"
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac':
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
else:
    raise NotADirectoryError
    
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4

for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"

    for snapnr in range(127,128,1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=True) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        

level   : 4
halo    : 24
snapnr  : 127
basedir : /hits/universe/GigaGalaxy/level4_MHD/
halodir : /hits/universe/GigaGalaxy/level4_MHD/halo_24/
snappath: /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/

[ 34.42281723  33.16259384  37.29567337]
Found 1783163 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02408556640148163
redshift: 2.220446049250313e-16
time    : 0.9999999999999998
center  : [ 0.  0.  0.]



In [8]:
filename = '/hits/universe/GigaGalaxy/level4_MHD/lists/halo_24starID_accreted_all_newmtree.dat'

fin = open(filename, "rb")
nacc = struct.unpack( 'i', fin.read(4) )[0]
idacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
subacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
snapacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
aflag = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)))
fofflag = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
rootid = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
pkmassid = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
fin.close()


- aflag  can be (0,1) for accreted stars, or those in an existing subhalo
- rootid is the unique progenitor ID of the subhalo in which the star was born
- pkmassid is the unique progenitor ID of the subhalo that a particle has been in when it was at it's maximum mass before it merged
- idacc is the particle ID
- subacc is the subfind ID of the subhalo is was born in
- snapacc is the snapshot is was born in


In [13]:
idacc

array([8796089319555, 6258063395953, 6258063445764, ..., 8796309435311,
       8796310808968, 8796292272677])

In [19]:
pkmassid

array([  219, 10243, 10243, ...,   247,   247,  9666])

In [23]:
def return_pkmassprogid_mostmassive(pkmassid, n=10):
    first_prog = np.array(list(sorted(set(list(pkmassid)))))
    stars_in_subhalo = np.zeros(len(first_prog))
    for i, pid in zip(range(len(first_prog)),first_prog):
        stars_in_subhalo[i] = sum( pkmassid==pid )
    nsort = np.argsort( stars_in_subhalo )[::-1]
    stars_in_subhalo = stars_in_subhalo[nsort].astype('int')
    first_prog = first_prog[nsort]

    return first_prog[:n]

In [25]:
first_prog = return_pkmassprogid_mostmassive(pkmassid)
#pindy, = np.where( (pkmassid==first_prog[0]) )


In [26]:
first_prog

array([  247,   219, 10243, 38592, 10245, 38673, 22288, 22289,   169, 24829])

In [27]:
pindy, = np.where( (pkmassid==first_prog[0]) )

In [28]:
pindy

array([    98,    138,    207, ..., 358658, 358659, 358660])